# **TVM : Tensor Virtual Machine**

In [1]:
!pip install apache-tvm
!pip install numpy
!pip install logger

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1758 sha256=c533d73f9f450bcaaddcbc1bf95e396185e14230cba0e972a731c176f66bf19f
  Stored in directory: /root/.cache/pip/wheels/fb/19/7b/09fc73f7503166eaf7f31b4aa0095b7f78af2ec0898e1f8312
Successfully built logger


In [2]:
import logging
import tvm
from tvm import relay
from tvm.contrib import graph_executor
from pathlib import Path


In [3]:
logging.basicConfig(level="DEBUG")
logger = logging.getLogger(__name__)
global input_shape

def get_tvm_targets(targets):
    tvm_targets = []
    transforms = []
    for target in targets:
        if "llvm" in target:
            tvm_targets.append(tvm.target.Target(target))
        else:
            raise ValueError("Unknown tvm target:", target)
    return tvm_targets, transforms


def convert_tvm(model, deploy_cfg):
    logger.info("Start onnx2tvm")
    print("start tvm convertion")
    model = os.path.abspath(model)
    print(os.path.basename(model).replace(".onnx", ".tar"))
    tvm_config = deploy_cfg["tvm_config"]
    tvm_config["out"] = os.path.basename(model).replace(".onnx", ".tar")
    original_workdir = os.getcwd()
    print(f"original workdir = {original_workdir}")
    onnx2tvm_workdir = os.path.join(os.getcwd(), "tvm")
    if Path(onnx2tvm_workdir).exists():
        shutil.rmtree(onnx2tvm_workdir)
    Path(onnx2tvm_workdir).mkdir(parents=True, exist_ok=True)
    os.chdir(onnx2tvm_workdir)
    stderr = os.dup(sys.stderr.fileno())
    log_stderr = open("onnx2tvm_stderr.txt", "wb")
    os.dup2(log_stderr.fileno(), sys.stderr.fileno())
    print("running conversion")
    p = Process(target=_convert, args=(model, deploy_cfg, onnx2tvm_workdir))
    p.start()
    p.join(timeout=tvm_config["timeout"])
    log_stderr.close()
    os.dup2(stderr, sys.stderr.fileno())
    with open("onnx2tvm_stderr.txt") as f:
        print(f.read())
    sys.stderr.flush()
    os.chdir(original_workdir)
    if p.is_alive():
        p.terminate()
        p.join()
        raise TimeoutError(
            f"TVM model convert: timeout after {tvm_config['timeout']} sec"
        )
    if p.exitcode is None or p.exitcode > 0:
        raise RuntimeError("TVM compile failed:", p.exitcode)
    tvm_config = deploy_cfg["tvm_config"]
    deploy_cfg[model[0]] = os.path.join(onnx2tvm_workdir, tvm_config["out"])

    if not os.path.exists(deploy_cfg[model[0]]):
        raise RuntimeError(f"TVM model didn't generated to {deploy_cfg.model[0]}")

    logger.info(
        "Successfully exported TVM model for %s: %s",
        tvm_config["compiler"],
        model,
    )
    print("finished tvm convertion")


def _convert(model, deploy_cfg, onnx2tvm_workdir: str):
    assert Path(model).exists(), model

    tvm_config = deploy_cfg["tvm_config"]

    onnx_model = onnx.load(model)
    onnx_input = onnx_model.graph.input[0]
    input_name = onnx_input.name
    input_shape = [d.dim_value for d in onnx_input.type.tensor_type.shape.dim]

    shape_dict = {input_name: input_shape}
    mod, params = relay.frontend.from_onnx(
        onnx_model, shape_dict, convert_config={"no_ort_dequantize": True}
    )
    with open("tvm_onnx_model_relay.txt", "w") as f:
        print(mod, file=f)

    targets, transforms = get_tvm_targets(
        tvm_config["targets"]
    )
    with tvm.transform.PassContext(opt_level=tvm_config["opt_level"]):
        relay.backend.te_compiler.get().clear()
        mod = tvm.transform.Sequential(COMMON_TRANSFORMS)(mod)
        for fn in transforms:
            mod = fn(mod)
        with open(f"tvm_{tvm_config['compiler']}_model_relay.txt", "w") as f:
            print(mod, file=f)
    lib = relay.build(mod, target=targets, params=params)
    lib.export_library(os.path.join(onnx2tvm_workdir, tvm_config["out"]))
    print(os.path.join(onnx2tvm_workdir, tvm_config["out"]))
    return lib


In [4]:
deploy_cfg = {
    "tvm_config": {
        "compiler": "llvm",
        "targets": ["llvm"],
        "opt_level": 3,
        "timeout": 300,
        "Model_onnx": "/kaggle/working/yolov5/yolov5n_fp32_int8.onnx",
        "image_dir": "/kaggle/working/datasets/coco/images/val2017/"
    }
}


In [5]:
model_path = "/kaggle/working/yolov5/yolov5n_fp32_int8.onnx"
convert_tvm(model_path, deploy_cfg)

start tvm convertion


NameError: name 'os' is not defined

## **Akarshit's Code**

In [ ]:
import torchvision.datasets as datasets

In [ ]:
test_dataset = datasets.ImageFolder(root=data_folder, transform=transform)
    subset_size=1000
    # Create a DataLoader for the test dataset
    subset_indices = list(range(0, 50000, 50))
    subset_dataset = Subset(test_dataset, subset_indices)
    test_loader = DataLoader(subset_dataset, batch_size=1, shuffle=False)


In [ ]:
def eval_tvm(args, data_loader):
    input_shape = (1, 3, 640, 640) #changed
    model_path=args.model+"_int8"+".onnx" #changed
    onnx_model=onnx.load(model_path)
    input_name = "x.1" #changed
    shape_dict = {input_name: input_shape}
    mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)
    target = "llvm"
    with tvm.transform.PassContext(opt_level=3):
        executor = relay.build_module.create_executor(
            "graph", mod, tvm.cpu(0), target, params
        ).evaluate()

    print("finished tvm convertion")

    x,y = next(iter(data_loader))
    ndarray = x.numpy()
    input_data = tvm.nd.array(ndarray.astype("float32"))
    start_time = time.time()
    output = executor(input_data).numpy()
    end_time = time.time()
    inference_time = end_time - start_time
    print("Inference Time:", inference_time, "seconds")
    top1_correct = 0
    top5_correct = 0
    total_samples = 0
    #acc check
    for idx, (images, labels) in tqdm(enumerate(data_loader), total=1000, desc="Processing images"):
        # Set the input data
        numpy_images = images.numpy()
        input_data = tvm.nd.array(numpy_images.astype("float32"))
        tvm_output = executor(input_data).numpy()
        predicted_labels = np.argmax(tvm_output, axis=1)
        top1_correct += np.sum(predicted_labels == labels.numpy())
        # Calculate top-5 accuracy
        top5_predicted_labels = np.argsort(tvm_output, axis=1)[:, -5:]
        for i in range(labels.size(0)):
            if labels.numpy()[i] in top5_predicted_labels[i]:
                top5_correct += 1

        total_samples += labels.size(0)
        if idx >= 1000:
            break

    # Calculate accuracy
    top1_accuracy = top1_correct / total_samples
    top5_accuracy = top5_correct / total_samples

    print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
    print(f"Top-5 Accuracy: {top5_accuracy * 100:.2f}%")

